In [ ]:
#AIM: To build Memory based chat system
#create an object where i can store my messages
#update this object with user message and AI reponse (write a function which can update our memory object)
#create a chain (pt, model and parser)
#In order to pass history to my chain, i need to make my PT accordingly which can accept history and new message
#whenever i call my chain, i should also invoke my memory method: so lets make a method which include chain invokation and memory call


In [6]:
%pip install -qU langchain_core langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 7.8 MB/s eta 0:00:00


In [20]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

# Initialize memory (list of messages)
memory = []

# Define the prompt template with the variables to replace
template = """
You are a helpful assistant. Here is the conversation history so far:
{history}

Now, respond to the following question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [21]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')

In [22]:
# Create a function to update memory with each new message
def update_memory(memory, question, response):
    memory.append(f"User: {question}")
    memory.append(f"Assistant: {response}")
    return memory

In [ ]:
model = ChatGroq()
output_parser = StrOutputParser()
conversation_chain = prompt | model | output_parser

In [19]:
def get_response(question, memory):

    # stringify the history of messages
    history = "\n".join(memory)

    # pass the history and the question to the chain
    result = conversation_chain.invoke({
        "history": history,
        "question": question
    })

    memory = update_memory(memory, question, result)

    return result, memory


In [ ]:

# Run the conversation
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    response, memory  = get_response(user_input, memory)
    print("Assistant:", response)